## Data-Driven Protein Engineering 
- Use of a machine learning approach to use additional data to support the design of novel enzymes
- Focus on a training a small dataset for 1st model then to be used on secondary dataset for model improvement

## Module Imports


In [335]:
'Import Libraries'

#Python Packages
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

#Dupont Packages
from prolerep.analysis.utils import read_csv_from_uri
import prolerep.analysis.utils as utils
import dotenv
dotenv.load_dotenv(".env")

#Scikit Packages
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler\

#Tensorflow Packages

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, activations
from tensorflow.keras.layers import Dense, Dropout, InputLayer, BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.initializers import RandomNormal, Constant
from tensorflow.keras.layers.experimental import preprocessing

## Load data
- *Upload df and select sequences (can be done with any particular dataframe)*

In [2]:
'Upload original dataset'
galapagos_stability_df = utils.read_csv_from_s3("s3://prolerep/datasets/galapagos_stability_df_reduced.csv") 
#galapagos_stability_df = galapagos_stability_df.reset_index()

In [3]:
galapagos_stability_df

,sequence,stability,temperature
0,-AQSVPWGISRVQAPAAH-NRGLRGSGVKVAVLDTGI-STHPDLNI...,0.00,50.0
1,-AQQVPYGVSQIKAPALH-EQGYTGQNVKVAVIDTGIDSSHPDLKV...,0.46,42.0
2,-AQSVPWGIRRVQAPAAH-NRGLTGSGVKVAVLDTGI-STHPDLNI...,0.34,46.0
3,-AQTVPWGISRVQAPAAH-NRGLTGAGVKVSVLDTGI-STHPDLNI...,0.95,45.0
4,-AQSVPYGVSQIKAPALH-SQGYTGSNVKVAVIDTGIDSSHPDLKV...,0.60,50.0
...,...,...,...
1995,-AQSVPWGISRVQAPAAH-NRGLTGSGVKVAVLDTGI-STHPDLNI...,0.02,45.0
1996,-AQQVPYGVSQIKAPALH-EQGYYGSNVKVAVIDSGIDSSHPDLKV...,0.29,42.0
1997,-AQSVPYGVSQIKAPALH-SQGYTGSNVKVAVIDTGIDSSHPDLKV...,0.66,46.0
1998,-AQQVPYGVEQIKAPALH-SQGYTGQNVKVAVIDTGIDSSHPDLKV...,0.06,60.0


In [4]:
'Obtain Protein Sequences'
galapagos_seqs = galapagos_stability_df['sequence'].str.replace("-","")

## Load embeddings

- *Load embedded sequences from .tfrecord file*

In [5]:
file = '/home/john/Dupont_Internship/Transfer_Learning/Galapagos_Data/my_galapagos_reduced_sequences.tfrecord'
parse_float_tensor = lambda x: tf.io.parse_tensor(x, out_type=tf.float64)
dataset = (
    tf.data.TFRecordDataset([file])
    .map(parse_float_tensor)
    .batch(len(galapagos_seqs)))

galapagos_embeddings = next(iter(dataset))
print(galapagos_embeddings.shape)

(2000, 3840)


### Create Features Dataframe

In [7]:
def create_dictionary(df, embeddings):
    
    '''Create a Dictionary with Multiple 
    Values paired to single key
    
    Returns: Dictionary with Multiple Values'''
    
    #Create needed arrays
    temperature = df['temperature']
    temp_array = np.array(temperature)
    embeddings_array = np.array(embeddings)

    embedded_dict = {}
    
    for index, (value1, value2) in enumerate(zip(embeddings_array, temp_array)):
        embedded_dict[index] = [value1, value2]

    return embedded_dict

In [8]:
def modify_df(original_df, embeddings):
    
    '''Create a dataframe with features to be used in NN
    
    Returns: Modified Dataframe'''
    
    embedded_dict = create_dictionary(original_df, embeddings)
    
    embeddings_list = []
    temperature_list = []
    for i in embedded_dict.values():
        embeddings_list.append(i[0])
        temperature_list.append(i[1])
    
    index = [str(i) for i in range(0, len(embeddings_list))]
    features_df = pd.DataFrame(embeddings_list, index=index)
    features_df['temperature'] = temperature_list
    
    return features_df

In [9]:
embedded_dict = create_dictionary(galapagos_stability_df, galapagos_embeddings)
features_df = modify_df(galapagos_stability_df, galapagos_embeddings)

### Data Prep
- Preparing the data before modelling

In [10]:
'Split and Standardize the Dataset'
def data_prep(original_df, features_df):
    'Standardize The Data'
    x = features_df
    x = np.array(x)
    y = original_df['stability']
    y = y.values

    # Split Data in train and validation
    x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size = 0.25, random_state = 6)

    return x_train, x_valid, y_train, y_valid

In [96]:
x_train, x_valid, y_train, y_valid = data_prep(galapagos_stability_df, features_df)
x_train

array([[-1.98100138, -2.34005857, -2.09497738, ...,  2.47909617,
         1.24270701, 60.        ],
       [-1.99232936, -2.40292597, -1.94223857, ...,  2.60929155,
         1.25136018, 68.        ],
       [-1.88469315, -2.37894297, -1.84657502, ...,  2.43665838,
         1.19955122, 60.        ],
       ...,
       [-1.84089506, -2.74671292, -2.0166223 , ...,  1.92245877,
         2.11539054, 60.        ],
       [-1.84819281, -2.36005425, -1.66958201, ...,  2.35021377,
         1.24997282, 60.        ],
       [-2.01986623, -2.46019411, -1.93254495, ...,  2.61112523,
         1.30350792, 68.        ]])

In [23]:
# 'Subsetting Dataset'
# first_embeddings = galapagos_embeddings[:1000]
# first_df = galapagos_stability_df[:1000]
# second_embeddings = galapagos_embeddings[1000:]
# second_df = galapagos_stability_df[1000:]

### Building and Fitting the Model

In [323]:
def normalized_model():

    # import BatchNormalization
    #from keras.layers.normalization import BatchNormalization

    # instantiate model
    model = Sequential()

    # we can think of this chunk as the input layer
    model.add(layers.Dense(128, input_shape = (x_train.shape[1],)))    
    model.add(BatchNormalization())
    model.add(layers.Activation(activations.relu))

    # we can think of this chunk as the hidden layer    
    model.add(layers.Dense(128))    
    model.add(BatchNormalization())
    model.add(layers.Activation(activations.relu))

    # we can think of this chunk as the output layer
    model.add(Dense(1))
    #model.add(layers.Activation(activations.softplus))
    
    # setting up the optimization of our weights 
    model.compile(loss='mean_squared_error', optimizer = Adam(lr = 0.001), metrics=['mean_absolute_error'])
    
    # ANN Summary
    model.summary()
    
    return model

In [324]:
model = normalized_model()
history = model.fit(x_train, y_train, batch_size = 25,
                    epochs = 50, verbose = 0 ,validation_data = (x_valid, y_valid))

Model: "sequential_43"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_115 (Dense)            (None, 128)               491776    
_________________________________________________________________
batch_normalization_64 (Batc (None, 128)               512       
_________________________________________________________________
activation_15 (Activation)   (None, 128)               0         
_________________________________________________________________
dense_116 (Dense)            (None, 128)               16512     
_________________________________________________________________
batch_normalization_65 (Batc (None, 128)               512       
_________________________________________________________________
activation_16 (Activation)   (None, 128)               0         
_________________________________________________________________
dense_117 (Dense)            (None, 1)               

In [268]:
def build_model():
    # Initialising the ANN
    model = Sequential()

    # Adding the input layer and the first hidden layer
    model.add(Dense(units = 128, activation = 'relu', input_shape = (x_train.shape[1],)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    # Adding the second hidden layer
    model.add(Dense(units = 256, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
   
    # Adding the output layer
    model.add(Dense(units = 1))

    # Compile the ANN
    model.compile(loss='mean_squared_error', optimizer = Adam(lr = 0.001), metrics=['mean_absolute_error'])

    # ANN Summary
    model.summary()
    
    return model


In [357]:
def build_model():
    # Initialising the ANN
    model = Sequential()

    # Adding the input layer and the first hidden layer
    model.add(Dense(units = 128, activation = 'relu', input_shape = (x_train.shape[1],)))
    model.add(BatchNormalization())
    #model.add(Dropout(0.2))
    
    # Adding the second hidden layer
    model.add(Dense(units = 128, activation = 'relu'))
    model.add(BatchNormalization())
    #model.add(Dropout(0.3))

    # Adding the output layer
    model.add(Dense(units = 1, activation = 'softplus'))

    # Compile the ANN
    model.compile(loss='mean_squared_error', optimizer = Adam(lr = 0.001), metrics=['mean_absolute_error'])

    # ANN Summary
    model.summary()
    
    return model


In [ ]:
#Call Model
model = build_model()
#Train Model
history = model.fit(x_train, y_train, batch_size = 25, epochs = 100, verbose = 0 ,validation_data = (x_valid, y_valid))

Model: "sequential_47"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_126 (Dense)            (None, 128)               491776    
_________________________________________________________________
batch_normalization_71 (Batc (None, 128)               512       
_________________________________________________________________
dense_127 (Dense)            (None, 128)               16512     
_________________________________________________________________
batch_normalization_72 (Batc (None, 128)               512       
_________________________________________________________________
dense_128 (Dense)            (None, 1)                 129       
Total params: 509,441
Trainable params: 508,929
Non-trainable params: 512
_________________________________________________________________


In [ ]:
# # This callback will stop the training when there is no improvement in the validation loss for 10 consecutive epochs.

# callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# history = model.fit(x_train, y_train, epochs=30, batch_size=50, callbacks=[callback], 
# verbose=1, validation_data = (x_valid, y_valid))

### Model Evaluation 

In [ ]:
#Evaluate the Model
score = model.evaluate(x_valid, y_valid, verbose=1)
    
print('Score loss:', score)

print ('Test loss:', round(score[0], 3))
print ('Test MAE:', round(score[1], 3))

In [ ]:
#Plot Interval Size Distribution
plt.figure(figsize=(16,8))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss', fontsize = 30)
plt.ylabel('Loss', fontsize = 25)
plt.xlabel('Epoch No.',  fontsize = 25)
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
y_pred = model.predict(x_valid)
#y_pred

In [ ]:
plt.figure(figsize=(16,8))
plt.scatter(y_valid, y_pred)
plt.suptitle('Model Predictions', fontsize=30)
plt.plot([y_valid.min(), y_valid.max()], [y_valid.min(), y_valid.max()], 'k--', lw=4)
plt.xlabel('True', fontsize = 25)
plt.ylabel('Predictions', fontsize = 25)

plt.show()


In [ ]:
'Model Predictions '
plt.figure(figsize=(16,8))
plt.plot(y_valid, color = 'red', label = 'Real data')
plt.plot(y_pred, color = 'blue', label = 'Predicted data')
plt.title('Prediction')
plt.legend()
plt.show()

In [235]:
# 'Error Distribution'
# plt.figure(figsize=(16,8))
# error = y_pred - y_valid
# plt.hist(error, bins = 25)
# plt.xlabel("Prediction Error")
# _ = plt.ylabel("Count")

### Save and Load Model

In [60]:
def save_nn_model(model, name):
    """
    Saves the model and weights to current local directory.

    :param model: the model of the network
    :param name: the name of the files used
    :return:
    """
    model_json = model.to_json()
    with open(name + '.json', 'w') as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(name + '.h5')
    print("Saved model to drive")

In [61]:
def load_nn_model(name):
    """
    Loads the saved model.

    :param name: the name of the files
    :return: the loaded model
    """
    json_file = open(name + '.json', 'r')
    model_json = json_file.read()
    json_file.close()
    model = model_from_json(model_json)
    # load weights into new model
    model.load_weights(name + '.h5') #corresponds to saved model above
    
    return model